# Redes neuronales recurrentes (RNN, LSTM, GRU)

Adaptado por: Jonnatan Arias Garcia

jariasg@uniquindio.edu.co

Jonnatan.arias@utp.edu.co

Machine Learning - 2024
Uniquindio - UTP

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

# shakespeare (generador de secuencias de caracteres por shakespeare)

In [2]:
enlace = 'https://rawcdn.githack.com/moona740/Nat_Moore_MA_Thesis/ffd9d46fbc034042e26eae25d65f0e98f9418b6c/pf_1.txt'

In [3]:
!wget -O /content/sample_data/pf_1.txt https://rawcdn.githack.com/moona740/Nat_Moore_MA_Thesis/ffd9d46fbc034042e26eae25d65f0e98f9418b6c/pf_1.txt

--2024-04-21 01:21:55--  https://rawcdn.githack.com/moona740/Nat_Moore_MA_Thesis/ffd9d46fbc034042e26eae25d65f0e98f9418b6c/pf_1.txt
Resolving rawcdn.githack.com (rawcdn.githack.com)... 104.21.234.231, 104.21.234.230, 2606:4700:3038::6815:eae7, ...
Connecting to rawcdn.githack.com (rawcdn.githack.com)|104.21.234.231|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/moona740/Nat_Moore_MA_Thesis/ffd9d46fbc034042e26eae25d65f0e98f9418b6c/pf_1.txt [following]
--2024-04-21 01:21:55--  https://raw.githubusercontent.com/moona740/Nat_Moore_MA_Thesis/ffd9d46fbc034042e26eae25d65f0e98f9418b6c/pf_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1149443 (1.1M) [text/plain]
Saving to: ‘/content/s

## Data inspección

In [4]:
text = open('/content/sample_data/pf_1.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1140435 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿<|startoftext|>
dog bone, stapler,
cribbage board, garlic press
because this window is loose—lacks
suction, lacks grip.
bungee cord, bootstrap,
dog leash, leather belt
because this window had sash cords.
they frayed. they broke.
feather dus


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))
print(vocab)

121 unique characters
['\n', '\r', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'E', 'I', 'J', 'M', 'R', 'S', 'T', 'V', 'W', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '\xa0', '¢', '\xad', '½', 'à', 'á', 'â', 'ä', 'å', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ï', 'ñ', 'ó', 'ö', 'ù', 'ú', 'ü', 'ā', 'đ', 'ę', 'ł', 'ū', 'ǫ', '́', 'ắ', 'ế', 'ố', 'ồ', '\u200b', '–', '—', '‘', '’', '“', '”', '•', '…', '\u2028', '\u2029', '\u2060', '\ufeff']


## Procesamiento del texto

### Vectorizar el texto

Antes de entrenar, necesitamos la cadrena a su representacion numerica. creando dos mapas, uno mapea caracteres a numeros y otra numeros a caracteres

In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])


mapeamos cada representacion a caracter, note que mapeamos con indices de 0 a len(unique)

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
#print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))
print(text[1], text[2], text[3], text[4], text[5], text[6], text[7], text[8], text[9], text[10], text[11], text[12], text[13])
print(text_as_int[1], text_as_int[2], text_as_int[3], text_as_int[4], text_as_int[5], text_as_int[6], text_as_int[7], text_as_int[8], text_as_int[9], text_as_int[10], text_as_int[11], text_as_int[12], text_as_int[13])
print(char2idx.get('t'))


< | s t a r t o f t e x t
30 74 66 67 48 65 67 62 53 67 52 71 67
67


### Tarea de predicción

Dado un carácter, o una secuencia de caracteres,

¿cuál es el siguiente carácter más probable? Esta es la tarea para la cual estamos entrenando al modelo.

La entrada al modelo será una secuencia de caracteres, y entrenamos al modelo para predecir la salida, es decir, el siguiente carácter en cada paso de tiempo.

Dado que las RNN mantienen un estado interno que depende de los elementos vistos previamente, ¿dado todos los caracteres computados hasta este momento, cuál es el siguiente carácter?

### Crear ejemplos de entrenamiento y objetivos.
A continuación, divida el texto en secuencias de ejemplo.

Cada secuencia de entrada contendrá seq_length caracteres del texto.

Para cada secuencia de entrada, los objetivos correspondientes contienen la misma longitud de texto, excepto desplazado un carácter hacia la derecha.

Por lo tanto, divida el texto en fragmentos de seq_length+1. Por ejemplo, digamos que seq_length es 4 y nuestro texto es "Hello". La secuencia de entrada sería "Hell" y la secuencia objetivo "o".

Para hacer esto, primero use la función tf.data.Dataset.from_tensor_slices para convertir el vector de texto en un flujo de índices de caracteres.

In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1) # // = floor division, rounds down to nearest whole number
print(examples_per_epoch)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

11291
﻿
<
|
s
t


El método de lotes nos permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeff<|startoftext|>\r\ndog bone, stapler,\r\ncribbage board, garlic press\r\nbecause this window is loose—lack'
's\r\nsuction, lacks grip.\r\nbungee cord, bootstrap,\r\ndog leash, leather belt\r\nbecause this window had sa'
"sh cords.\r\nthey frayed. they broke.\r\nfeather duster, thatch of straw, empty\r\nbottle of elmer's glue\r\n"
'because this window is loud—its hinges clack\r\nopen, clack shut.\r\nstuffed bear, baby blanket,\r\nsingle '
"crib newel\r\nbecause this window is split. it's dividing\r\nin two.\r\nvelvet moss, sagebrush,\r\nwillow bra"


Para cada secuencia, duplíquela y desplácela para formar el texto de entrada y de destino utilizando el método de mapeo para aplicar una función simple a cada lote:

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
tf.print(dataset)

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


Imprimir los primeros ejemplos de valores de entrada y objetivo:

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

print(dataset.take(1))

Input data:  '\ufeff<|startoftext|>\r\ndog bone, stapler,\r\ncribbage board, garlic press\r\nbecause this window is loose—lac'
Target data: '<|startoftext|>\r\ndog bone, stapler,\r\ncribbage board, garlic press\r\nbecause this window is loose—lack'
<_TakeDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


Cada índice de estos vectores se procesa como un paso de tiempo. Para la entrada en el paso de tiempo 0, el modelo recibe el índice de "F" e intenta predecir el índice de "i" como el siguiente carácter. En el siguiente paso de tiempo, hace lo mismo pero la RNN considera el contexto del paso anterior además del carácter de entrada actual.

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 120 ('\ufeff')
  expected output: 30 ('<')
Step    1
  input: 30 ('<')
  expected output: 74 ('|')
Step    2
  input: 74 ('|')
  expected output: 66 ('s')
Step    3
  input: 66 ('s')
  expected output: 67 ('t')
Step    4
  input: 67 ('t')
  expected output: 48 ('a')


### Crear lotes de entrenamiento

Utilizamos tf.data para dividir el texto en secuencias manejables. Pero antes de alimentar estos datos al modelo, necesitamos mezclar los datos y empaquetarlos en lotes.

In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Construir el modelo

Utilice tf.keras.Sequential para definir el modelo. Para este ejemplo simple, se utilizan tres capas para definir nuestro modelo:
* tf.keras.layers.Embedding: La capa de entrada. Una tabla de búsqueda entrenable que asignará los números de cada carácter a un vector con dimensiones de embedding_dim;
* tf.keras.layers.GRU: Un tipo de RNN con unidades de tamaño rnn_units (también puede usar una capa LSTM aquí)
* tf.keras.layers.Dense: La capa de salida, con vocab_size salidas.

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 2048

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           30976     
                                                                 
 lstm (LSTM)                 (64, None, 2048)          18882560  
                                                                 
 dense (Dense)               (64, None, 121)           247929    
                                                                 
Total params: 19161465 (73.10 MB)
Trainable params: 19161465 (73.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Para cada carácter, el modelo busca la incrustación, ejecuta el GRU un paso de tiempo con la incrustación como entrada y aplica la capa densa para generar digitos que predicen la probabilidad logarítmica del siguiente carácter.


## Pruebe el modelo
Ahora ejecute el modelo para ver que se comporte como se espera.
Primero verifique la forma de la salida:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 121) # (batch_size, sequence_length, vocab_size)


En el ejemplo anterior, la longitud de la secuencia de entrada es de 100, pero el modelo se puede ejecutar en entradas de cualquier longitud:

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           30976     
                                                                 
 lstm (LSTM)                 (64, None, 2048)          18882560  
                                                                 
 dense (Dense)               (64, None, 121)           247929    
                                                                 
Total params: 19161465 (73.10 MB)
Trainable params: 19161465 (73.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Para obtener predicciones reales del modelo, necesitamos muestrear de la distribución de salida, para obtener los índices reales de los caracteres. Esta distribución está definida por los logits sobre el vocabulario de caracteres. Nota: es importante muestrear de esta distribución, ya que tomar el argmax de la distribución puede hacer que el modelo se quede fácilmente atrapado en un bucle. Pruébalo para el primer ejemplo en el lote:

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

Esto nos da, en cada paso de tiempo, una predicción del índice del próximo carácter.

In [22]:
sampled_indices

array([ 40, 104,   8,  98,  48,  35,  43,  61, 107,   8,  23,  71,  43,
        28,  39,  53,   4,  35,  98,  34,  99,  65, 115,   3,  75,  63,
        30,  85,  42, 119,  76,   3,  72,  31,  76,  96,  15,  25,  20,
        96,  47,  70, 120, 110,  48,  69,  22,  51, 116,  13,  98,  82,
        73,  53,  24,   1,  46,  67,   8,  50, 107,  96,  64,  34,  94,
        45,  56, 112,  44,  32,  11,  17, 100,  44,  87,  38, 113,  12,
        44,  26, 119,   3,  21,  48,  62,  99,  61,  37,  11,  32,  54,
        89, 115,  19,   0,  77, 118, 109,  89,  81])

Decodifica esto para ver el texto predicho por este modelo no entrenado.

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ouds go, nevertheless,\r\nin their direction.\r\n<|endoftext|>\r\n<|startoftext|>\r\ncapitán profundo, capit'

Next Char Predictions: 
 'Sắ&đaEWnồ&5xW:Rf"EđAęr•!\xa0p<èV\u2060¢!y=¢ü-72ü`w\ufeff—av4d…+đäzf6\r_t&cồüqAù]i’[>)/ł[êM“*[8\u2060!3aoęnJ)>gí•1\n\xad\u2029–íâ'


## Entrena el modelo


En este punto, el problema se puede tratar como un problema de clasificación estándar. Dado el estado RNN anterior y la entrada en este paso de tiempo, predice la clase del próximo carácter.

### Adjunte un optimizador y una función de pérdida.

La función de pérdida estándar tf.keras.losses.sparse_categorical_crossentropy funciona en este caso porque se aplica a lo largo de la última dimensión de las predicciones. Dado que nuestro modelo devuelve logit, necesitamos establecer el indicador from_logits.

In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 121)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.7958283


Configure el procedimiento de entrenamiento utilizando el método tf.keras.Model.compile. Utilizaremos tf.keras.optimizers.Adam con argumentos predeterminados y la función de pérdida.

In [25]:
model.compile(optimizer='adam', loss=loss)

### Configurar puntos de control

Utilice un tf.keras.callbacks.ModelCheckpoint para asegurarse de que los puntos de control se guarden durante el entrenamiento:

In [26]:
%ls # Directory where the checkpoints will be saved
checkpoint_dir = '/sample_data/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

sample_data/


### Ejecutar el entrenamiento

Para mantener un tiempo de entrenamiento razonable, utilice 10 épocas para entrenar el modelo. En Colab, configure el tiempo de ejecución en GPU para un entrenamiento más rápido.

In [27]:
history = model.fit(dataset, epochs=2, callbacks=[checkpoint_callback])

Epoch 1/2
176/176 [==============================] - 441s 2s/step - loss: 2.5975
Epoch 2/2
176/176 [==============================] - 442s 3s/step - loss: 1.9610


In [28]:
from keras import backend as K
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])
layer_output = get_3rd_layer_output
print(layer_output)

<function function.<locals>.func at 0x7defc3d4d480>


## Generar texto

### Restaurar el último punto de control

Para mantener este paso de predicción simple, use un tamaño de lote de 1. Debido a la forma en que se transmite el estado de la RNN de paso a paso, el modelo solo acepta un tamaño de lote fijo una vez construido. Para ejecutar el modelo con un batch_size diferente, necesitamos reconstruir el modelo y restaurar los pesos desde el punto de control.

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'/sample_data/training_checkpoints/ckpt_2'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            30976     
                                                                 
 lstm_1 (LSTM)               (1, None, 2048)           18882560  
                                                                 
 dense_1 (Dense)             (1, None, 121)            247929    
                                                                 
Total params: 19161465 (73.10 MB)
Trainable params: 19161465 (73.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Bucle de predicción

El siguiente bloque de código genera el texto:
* Comienza eligiendo una cadena de inicio, inicializando el estado RNN y estableciendo el número de caracteres a generar.

* Obtén la distribución de predicción del siguiente carácter usando la cadena de inicio y el estado RNN.

* Luego, utiliza una distribución categórica para calcular el índice del carácter predicho. Utiliza este carácter predicho como nuestra siguiente entrada al modelo.

* El estado RNN devuelto por el modelo se retroalimenta al modelo para que ahora tenga más contexto, en lugar de solo un carácter. Después de predecir el siguiente carácter, los estados RNN modificados se vuelven a retroalimentar al modelo, que es cómo aprende a medida que obtiene más contexto de los caracteres predichos anteriormente.

Al observar el texto generado, verás que el modelo sabe cuándo capitalizar, hacer párrafos e imitar un vocabulario de escritura similar al de Shakespeare. Con el pequeño número de épocas de entrenamiento, aún no ha aprendido a formar frases coherentes.

In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 500.0

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.45

  # Here batch size == 1
  model.reset_states()
  for i in range(int(num_generate)):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string="This "))

This where i lead for the word the was rome in the marr of a planted strains in the recting the minds
the propers and beling it me be a stread of the crose in the livers
that conds all dister bedain prosed
strees and fire and street in a pitcenting what its fires and clades on the cares and for a sain, the cand
with the caret and gring and cire of the strands,
the carred of the pirlow the was and who wat i placks and mast
ald and the crocked the bread of the sund sind like the latters
her side
